# 避免数据重复训练，这里我们采用pickle

In [102]:
import numpy as np
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(shm_size_mb=2,nb_workers=5,progress_bar=True,verbose=1)

import pickle
import matplotlib.pyplot as plt
data_path="C:/Users/dadia/Desktop/FastAI/vaccination_all_tweets.csv"
out_path="C:/Users/dadia/Desktop/FastAI/"


def write_pickle(obj_in,out_path_in,name_in):
    pickle.dump(obj_in,open(out_path_in+name_in+".pk","wb"))
    
def read_pickle(path_in, file_name):
    tmp=pickle.load(open(path_in + file_name + ".pk", "rb"))
    return tmp


In [103]:
data=pd.read_csv(data_path)
data.head()

id             user_name              user_location  \
0  1340539111971516416            Rachel Roh  La Crescenta-Montrose, CA   
1  1338158543359250433           Albert Fong          San Francisco, CA   
2  1337858199140118533              eli🇱🇹🇪🇺👌                   Your Bed   
3  1337855739918835717         Charles Adler     Vancouver, BC - Canada   
4  1337854064604966912  Citizen News Channel                        NaN   

                                    user_description         user_created  \
0  Aggregator of Asian American news; scanning di...  2009-04-08 17:52:46   
1  Marketing dude, tech geek, heavy metal & '80s ...  2009-09-21 15:27:30   
2                                     heil, hydra 🖐☺  2020-06-25 23:30:28   
3  Hosting "CharlesAdlerTonight" Global News Radi...  2008-09-10 11:28:53   
4  Citizen News Channel bringing you an alternati...  2020-04-23 17:58:42   

   user_followers  user_friends  user_favourites  user_verified  \
0             405          1692             3247          False   
1             834           666              178          False   
2              10            88              155          False   
3           49165          3933            21853           True   
4             152           580             1473          False   

                  date                                               text  \
0  2020-12-20 06:06:44  Same folks said daikon paste could treat a cyt...   
1  2020-12-13 16:27:13  While the world has been on the wrong side of ...   
2  2020-12-12 20:33:45  #coronavirus #SputnikV #AstraZeneca #PfizerBio...   
3  2020-12-12 20:23:59  Facts are immutable, Senator, even when you're...   
4  2020-12-12 20:17:19  Explain to me again why we need a vaccine @Bor...   

                                            hashtags               source  \
0                                 ['PfizerBioNTech']  Twitter for Android   
1                                                NaN      Twitter Web App   
2  ['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...  Twitter for Android   
3                                                NaN      Twitter Web App   
4     ['whereareallthesickpeople', 'PfizerBioNTech']   Twitter for iPhone   

   retweets  favorites  is_retweet  
0         0          0       False  
1         1          1       False  
2         0          0       False  
3       446       2129       False  
4         0          0       False

In [104]:
#data.info()

In [105]:
data=data.dropna(subset=["text","date"])
#data=data.drop(labels=["user_name","user_description","user_created","user_favourites","hashtags","source","is_retweet","user_location","user_followers","user_friends","user_verified","date"],axis=1)
data.info()
#processed_docs=processed_docs["text"].drop_duplicates(keep='first', inplace=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69718 entries, 0 to 69717
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                69718 non-null  int64 
 1   user_name         69718 non-null  object
 2   user_location     53891 non-null  object
 3   user_description  64846 non-null  object
 4   user_created      69718 non-null  object
 5   user_followers    69718 non-null  int64 
 6   user_friends      69718 non-null  int64 
 7   user_favourites   69718 non-null  int64 
 8   user_verified     69718 non-null  bool  
 9   date              69718 non-null  object
 10  text              69718 non-null  object
 11  hashtags          54904 non-null  object
 12  source            69599 non-null  object
 13  retweets          69718 non-null  int64 
 14  favorites         69718 non-null  int64 
 15  is_retweet        69718 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 8.1+ MB


# 无聊的清理数据

In [106]:
def clean_text(str_in):
    #str_in=remove_non_ascii(str_in)
    import re
    str_in=re.sub(r"http\S+","",str_in) #url
    str_in=re.sub(r"@[A-Za-z0-9_]+","",str_in) #mention
    str_in=re.sub(r"#[A-Za-z0-9_]+","",str_in) #hashtag
    str_in="".join(re.sub(r"[^A-Za-z']+"," ",str_in).strip()) #other characters
    
    '''
    from nltk.corpus import stopwords
    sw=stopwords.words('english')
    proc_text = [word for word in str_in.split() if word not in sw] #sw filter
    str_in= ' '.join(proc_text)
    
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    proc_txt = [ps.stem(word) for word in str_in.split()] #stemming
    str_in= ' '.join(proc_txt)
    '''
    
    return str_in


In [107]:
clean_text("#CoronavirusVaccine #CoronaVaccine #CovidVaccine Australia is doing very good https://t.co/kBT7l6pArY")

'Australia is doing very good'

In [108]:
data["cleaned_text"]=data["text"].parallel_apply(clean_text)

In [109]:
data["cleaned_text"].head()

0    Same folks said daikon paste could treat a cyt...
1    While the world has been on the wrong side of ...
2             Russian vaccine is created to last years
3    Facts are immutable Senator even when you're n...
4            Explain to me again why we need a vaccine
Name: cleaned_text, dtype: object

In [110]:
write_pickle(data,out_path,"cleaned")

In [111]:
data=read_pickle(out_path,"cleaned")
data.head()

id             user_name              user_location  \
0  1340539111971516416            Rachel Roh  La Crescenta-Montrose, CA   
1  1338158543359250433           Albert Fong          San Francisco, CA   
2  1337858199140118533              eli🇱🇹🇪🇺👌                   Your Bed   
3  1337855739918835717         Charles Adler     Vancouver, BC - Canada   
4  1337854064604966912  Citizen News Channel                        NaN   

                                    user_description         user_created  \
0  Aggregator of Asian American news; scanning di...  2009-04-08 17:52:46   
1  Marketing dude, tech geek, heavy metal & '80s ...  2009-09-21 15:27:30   
2                                     heil, hydra 🖐☺  2020-06-25 23:30:28   
3  Hosting "CharlesAdlerTonight" Global News Radi...  2008-09-10 11:28:53   
4  Citizen News Channel bringing you an alternati...  2020-04-23 17:58:42   

   user_followers  user_friends  user_favourites  user_verified  \
0             405          1692             3247          False   
1             834           666              178          False   
2              10            88              155          False   
3           49165          3933            21853           True   
4             152           580             1473          False   

                  date                                               text  \
0  2020-12-20 06:06:44  Same folks said daikon paste could treat a cyt...   
1  2020-12-13 16:27:13  While the world has been on the wrong side of ...   
2  2020-12-12 20:33:45  #coronavirus #SputnikV #AstraZeneca #PfizerBio...   
3  2020-12-12 20:23:59  Facts are immutable, Senator, even when you're...   
4  2020-12-12 20:17:19  Explain to me again why we need a vaccine @Bor...   

                                            hashtags               source  \
0                                 ['PfizerBioNTech']  Twitter for Android   
1                                                NaN      Twitter Web App   
2  ['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...  Twitter for Android   
3                                                NaN      Twitter Web App   
4     ['whereareallthesickpeople', 'PfizerBioNTech']   Twitter for iPhone   

   retweets  favorites  is_retweet  \
0         0          0       False   
1         1          1       False   
2         0          0       False   
3       446       2129       False   
4         0          0       False   

                                        cleaned_text  
0  Same folks said daikon paste could treat a cyt...  
1  While the world has been on the wrong side of ...  
2           Russian vaccine is created to last years  
3  Facts are immutable Senator even when you're n...  
4          Explain to me again why we need a vaccine

# 多核处理数据，牛逼

In [112]:
def extract_senti(var_in):
    tmp = ""
    try:
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        senti_t = SentimentIntensityAnalyzer()
        tmp = senti_t.polarity_scores(var_in)
        tmp = tmp["compound"]
    except:
        pass
    return tmp

In [113]:
data["senti"]=data.cleaned_text.parallel_apply(extract_senti)

In [114]:
write_pickle(data[data["senti"]<-0.05],out_path,"negative_tweets")

In [115]:
data=read_pickle(out_path,"negative_tweets")
data

id              user_name  \
1      1338158543359250433            Albert Fong   
18     1337781178665816064  Braderz73🌹#GTTO 🇪🇺 🇬🇧   
19     1337776763384958976               Alex Vie   
23     1337773751841144833              BOOM Live   
25     1337770118017900550        Devan Surendran   
...                    ...                    ...   
69684  1382279249877344261     Nandakishore-డు, N   
69693  1382268491974135811       Rahul Shrivastav   
69694  1382268038062362627               editorji   
69706  1382254162855923712   Shashank Shekhar Jha   
69715  1382245076303179778         Robert ka Boss   

                     user_location  \
1                San Francisco, CA   
18                     Bristol, UK   
19                 Los Angeles, CA   
23                   Mumbai, India   
25             Nottingham, England   
...                            ...   
69684  HYDERABAD, (Bhagya-Nagaram)   
69693                Raipur, India   
69694         Sultanpur, New Delhi   
69706                 दिल्ली, भारत   
69715                          NaN   

                                        user_description         user_created  \
1      Marketing dude, tech geek, heavy metal & '80s ...  2009-09-21 15:27:30   
18     One of those lefty types, I believe in a faire...  2012-07-24 08:18:51   
19     Marine vet. Yogi. Krav Maga. Runner. Climber. ...  2010-01-24 04:43:57   
23     IFCN certified fact-driven journalism. India's...  2014-03-16 03:52:50   
25     NHS Doctor | Singer | Travel, nature and dog l...  2010-01-16 23:59:43   
...                                                  ...                  ...   
69684  Hyderabad for 99.9999% of life;\nAn insufferab...  2012-07-28 03:32:24   
69693  #political enthusiast#patriot #President #IT c...  2011-11-26 09:24:22   
69694  editorji is your curated short video news plat...  2017-08-13 14:29:09   
69706  Hindu | Lawyer | @goachronicle Editor (Legal) ...  2012-07-29 14:33:23   
69715  Lily don't be silly you want to know me just g...  2020-11-26 09:24:12   

       user_followers  user_friends  user_favourites  user_verified  \
1                 834           666              178          False   
18               6430          6292            45007          False   
19                125           442             5401          False   
23              64185          1183             1794           True   
25                116            86              268          False   
...               ...           ...              ...            ...   
69684             501          1617            29793          False   
69693             150           369             1743          False   
69694           33458            17              237           True   
69706            7600           194            23251          False   
69715              76            65               78          False   

                      date                                               text  \
1      2020-12-13 16:27:13  While the world has been on the wrong side of ...   
18     2020-12-12 15:27:42  .@Pfizer will rake in billions from its expens...   
19     2020-12-12 15:10:09  The trump administration failed to deliver on ...   
23     2020-12-12 14:58:11  The US Food and Drug Administration (FDA) has ...   
25     2020-12-12 14:43:45  No.1 of 2 done.\n\n#ThankYouNHS @NHSuk @MHRAgo...   
...                    ...                                                ...   
69684  2021-04-14 10:27:09  It is useless to try to shame @BJP4India leade...   
69693  2021-04-14 09:44:24  Shame on you\n@rsprasad \n#SputnikV https://t....   
69694  2021-04-14 09:42:36  After Russia’s #SputnikV received an expert pa...   
69706  2021-04-14 08:47:28  #SputnikV got clearance by DCGI.\n\nThird vacc...   
69715  2021-04-14 08:11:21  Hello, it s because of this stubbornness and f...   

                                                hashtags  \
1                                                    NaN   
18         

# 处理时间数据以防乱码

In [116]:
def process_time(str_in):
    from datetime import datetime
    try:
        a=datetime.strptime(str_in,"%Y/%m/%d %H:%M")
        return a
    except:
        try:
            b=datetime.strptime(str_in[:-3],"%Y/%m/%d %H:%M")
            return b
        except:
            try:
                c=datetime.strptime(str_in,"%d-%m-%Y %H:%M")
                return c
            except:
                try:
                    d=datetime.strptime(str_in[:-3],"%d-%m-%Y %H:%M")
                    return d
                except:
                    try:
                        e=datetime.strptime(str_in[:-3],"%Y-%m-%d %H:%M")
                        return e
                    except:
                        return datetime.strptime("1-1-1000 0:00","%m-%d-%Y %H:%M")

In [117]:
process_time("2020/10/1 10:24:56")

datetime.datetime(2020, 10, 1, 10, 24)

In [118]:
data["new_date"]=data.date.parallel_apply(process_time)

In [119]:
write_pickle(data,out_path,"date")

In [120]:
data=read_pickle(out_path,"date")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13500 entries, 1 to 69715
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                13500 non-null  int64         
 1   user_name         13500 non-null  object        
 2   user_location     10455 non-null  object        
 3   user_description  12538 non-null  object        
 4   user_created      13500 non-null  object        
 5   user_followers    13500 non-null  int64         
 6   user_friends      13500 non-null  int64         
 7   user_favourites   13500 non-null  int64         
 8   user_verified     13500 non-null  bool          
 9   date              13500 non-null  object        
 10  text              13500 non-null  object        
 11  hashtags          9753 non-null   object        
 12  source            13498 non-null  object        
 13  retweets          13500 non-null  int64         
 14  favorites         1350

In [121]:
data=data.sort_values(by=["new_date"])

In [122]:
data.iloc[:10]

id              user_name         user_location  \
44   1337727767551553536       Daily News Egypt                 Egypt   
36   1337735595704115200             John Berry                   NaN   
31   1337758104847572999          Isabel Winnwa       Berlin, Germany   
30   1337760271151063040            Andy Thomas                London   
25   1337770118017900550        Devan Surendran   Nottingham, England   
23   1337773751841144833              BOOM Live         Mumbai, India   
19   1337776763384958976               Alex Vie       Los Angeles, CA   
18   1337781178665816064  Braderz73🌹#GTTO 🇪🇺 🇬🇧           Bristol, UK   
292  1337889168895201280         Oigetit Health    Silicon Valley, CA   
288  1337899774788456448   Simon Hodes ⬅️2m➡️ 😷  GP Partner, Watford    

                                      user_description         user_created  \
44   Egypt's Only Daily Independent Newspaper in En...  2009-04-26 07:56:24   
36   English (not British or European) beer swillin...  2020-10-13 15:21:27   
31   Into (EU) politics, books, sports & all things...  2020-03-30 13:47:15   
30              Retweets not necessarily endorsements.  2009-03-07 20:39:15   
25   NHS Doctor | Singer | Travel, nature and dog l...  2010-01-16 23:59:43   
23   IFCN certified fact-driven journalism. India's...  2014-03-16 03:52:50   
19   Marine vet. Yogi. Krav Maga. Runner. Climber. ...  2010-01-24 04:43:57   
18   One of those lefty types, I believe in a faire...  2012-07-24 08:18:51   
292  Oigetit (Oh, I get it) filters Fake News. Try ...  2015-07-22 19:53:48   
288  NHS GP #Chronoptimist LMC Rep #TeamGP💙Trainer ...  2014-07-28 13:32:12   

     user_followers  user_friends  user_favourites  user_verified  \
44           278080           116              765           True   
36                2             0                0          False   
31              230           652             6098          False   
30             1151          4301            95963          False   
25              116            86              268          False   
23            64185          1183             1794           True   
19              125           442             5401          False   
18             6430          6292            45007          False   
292             952           894             2556          False   
288            1963          2429            18879          False   

                    date                                               text  \
44   2020-12-12 11:55:28  #FDA authorizes #PfizerBioNTech #coronavirus v...   
36   2020-12-12 12:26:34  The #FDA finally issues #EUA now comes the pro...   
31   2020-12-12 13:56:01  @sbnumb3 @nytimes Please don’t spread fake new...   
30   2020-12-12 14:04:37  @ZubyMusic 6 deaths so far. \nIt's only death,...   
25   2020-12-12 14:43:45  No.1 of 2 done.\n\n#ThankYouNHS @NHSuk @MHRAgo...   
23   2020-12-12 14:58:11  The US Food and Drug Administration (FDA) has ...   
19   2020-12-12 15:10:09  The trump administration failed to deliver on ...   
18   2020-12-12 15:27:42  .@Pfizer will rake in billions from its expens...   
292  2020-12-12 22:36:49  The announcement comes after federal authoriti...   
288  2020-12-12 23:18:57  @Janey94803852 @Maureenprsb Still seems unclea...   

                                              hashtags               source  \
44            ['FDA', 'PfizerBioNTech', 'coronavirus']      Twitter Web App   
36                    ['FDA', 'EUA', 'PfizerBioNTech']      Twitter Web App   
31                                                 NaN   Twitter for iPhone   
30                  ['CovidVaccines', 'Pfizervaccine']  Twitter for Android   
25   ['ThankYouNHS', 'PfizerBioNTech', 'CovidVaccine']      Twitter Web App   
23                                                 NaN      Twitter Web App   
19        ['COVIDIOTS', 'coronavirus', 'CovidVaccine']   Twitter for iPhone   
18                                    ['CovidVaccine']  Twitter for Android   
292    

In [123]:
data=data.iloc[9:]
data

id             user_name  \
288    1337899774788456448  Simon Hodes ⬅️2m➡️ 😷   
287    1337902845618774018       Fares Bou-Daher   
274    1337948680263909380                    NR   
264    1337974613280759808              LatestLY   
259    1337979102544850946           IANS Tweets   
...                    ...                   ...   
67913  1385233889044537345                  Amol   
67911  1385236168820854793     Workout Solutions   
67907  1385236820854812676     Workout Solutions   
67906  1385236855092822016     Workout Solutions   
67905  1385236941180973057     Workout Solutions   

                        user_location  \
288              GP Partner, Watford    
287                               NaN   
274            Kuala Lumpur, Malaysia   
264                     Mumbai, India   
259                         New Delhi   
...                               ...   
67913                             NaN   
67911  Toronto, Canada  and Worldwide   
67907  Toronto, Canada  and Worldwide   
67906  Toronto, Canada  and Worldwide   
67905  Toronto, Canada  and Worldwide   

                                        user_description         user_created  \
288    NHS GP #Chronoptimist LMC Rep #TeamGP💙Trainer ...  2014-07-28 13:32:12   
287    Direct with a conservative approach in serious...  2020-06-18 22:14:51   
274    PhD(Virology)/Academic(R)/RockefellerFdnFellow...  2009-04-11 14:59:38   
264    https://t.co/U4rHOvEUQR focuses on the What, W...  2017-02-07 05:41:28   
259              India's largest independent News Agency  2010-05-08 13:21:45   
...                                                  ...                  ...   
67913                                                NaN  2018-08-25 08:56:07   
67911  George Tsanis – Workout Solutions Health and F...  2010-09-20 17:01:08   
67907  George Tsanis – Workout Solutions Health and F...  2010-09-20 17:01:08   
67906  George Tsanis – Workout Solutions Health and F...  2010-09-20 17:01:08   
67905  George Tsanis – Workout Solutions Health and F...  2010-09-20 17:01:08   

       user_followers  user_friends  user_favourites  user_verified  \
288              1963          2429            18879          False   
287                18           106             3107          False   
274              1841          1657            48524          False   
264             67096           344              118           True   
259             59733            45              103           True   
...               ...           ...              ...            ...   
67913             214           658           118604          False   
67911            1200           236             2182          False   
67907            1200           236             2182          False   
67906            1200           236             2182          False   
67905            1200           236             2182          False   

                      date                                               text  \
288    2020-12-12 23:18:57  @Janey94803852 @Maureenprsb Still seems unclea...   
287    2020-12-12 23:31:10  Say NO to Covid-19 vaccines!\n#vaccine \n#vacc...   
274    2020-12-13 02:33:17  You're wrong: MALAYSIA has already pre-ordered...   
264    2020-12-13 04:16:20  Pfizer-BioNTech COVID-19 Vaccine Side Effects:...   
259    2020-12-13 04:34:11  Health #Canada has warned people allergic to a...   
...                    ...                                                ...   
67913  2021-04-22 14:07:50  World Always Underestimate "The Russia"😎 #Sput...   
67911  2021-04-22 14:16:53  #BREAKING: EMBARRASING Ontario Canada is repor...   
67907  2021-04-22 14:19:29  EMBARRASSING Ontario is reporting 3682  cases ...   
67906  2021-04-22 14:19:37  @fordnation EMBARRASSING Ontario is reporting ...   
67905  2021-04-22 14:19:57  @celliottability EMBARRASSING Ontario is repor...   

                                                hashtags               source  \
288                       

# 地点数据处理以防乱码

In [124]:
data=data.dropna(subset=["user_location"])

In [125]:
def preproc_loc(str_in):
    import re
    try:
        a=re.sub("[^A-Za-z']+", " ", str_in).lower()
        return set(a.split())
    except:
        pass

In [126]:
preproc_loc("Mumbai and Nagpur India")

{'and', 'india', 'mumbai', 'nagpur'}

In [127]:
data["cleaned_location"]=data.user_location.parallel_apply(preproc_loc)

C:\Users\dadia\AppData\Local\Temp/ipykernel_12304/1175550921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["cleaned_location"]=data.user_location.parallel_apply(preproc_loc)


In [128]:
data["cleaned_location"].head()

288       {gp, watford, partner}
274    {malaysia, kuala, lumpur}
264              {mumbai, india}
259                 {delhi, new}
248                 {delhi, new}
Name: cleaned_location, dtype: object

In [130]:
# data["cleaned_location"].to_csv("C:/Users/Matt/xuexi/machine_learning/"+"location.csv")

In [131]:
us=set(["us","usa","america","american","states","angeles","chicago","york",'alabama','alaska','arizona','arkansas','california','colorado','connecticut','delaware','florida','georgia','hawaii','idaho','illinois','indiana','iowa','kansas','kentucky','louisiana','maine','maryland','massachusetts','michigan','minnesota','mississippi','missouri','montana','nebraska','nevada','hampshire','jersey','york','carolina','dakota','ohio','oklahoma','oregon','pennsylvania','rhode','carolina','dakota','tennessee','texas','utah' 'vermont','virginia' 'washington','virginia','wisconsin','wyoming'])

india=set(['india','indian','hindu','mumbai', 'delhi', 'bengalore', 'hyderabad', 'ahmedabad', 'chennai', 'kolkata', 'surat', 'pune', 'jaipur','Andhra','Pradesh','andhra','pradesh', 'amaravati', 'arunachal','pradesh', 'itanagar', 'assam', 'dispur', 'bihar', 'patna', 'chhattisgarh', 'raipur', 'goa', 'panaji', 'gujarat', 'gandhinagar', 'haryana', 'chandigarh', 'himachal','pradesh', 'shimla', 'jharkhand', 'ranch', 'karnataka', 'bengaluru', 'kerala', 'thiruvananthapuram', 'madhya', 'pradesh', 'bhopal', 'maharashtra', 'mumbai', 'manipur', 'imphal','meghalaya', 'shillong', 'mizoram', 'aizawl', 'nagaland', 'kohima', 'odisha', 'bhubaneswar', 'punjab', 'chandigarh', 'rajasthan', 'jaipur', 'sikkim', 'gangtok', 'tamil','nadu','chennai', 'telangana', 'hyderabad', 'tripura', 'agartala', 'uttar', 'pradesh', 'lucknow', 'uttarakhand', 'dehradun', 'gairsain', 'summer', 'bengal','kolkata']    )

In [132]:
def country(set_in):
    us=set(["us","usa","america","american","states","angeles","chicago","york",'alabama','alaska','arizona','arkansas','california','colorado','connecticut','delaware','florida','georgia','hawaii','idaho','illinois','indiana','iowa','kansas','kentucky','louisiana','maine','maryland','massachusetts','michigan','minnesota','mississippi','missouri','montana','nebraska','nevada','hampshire','jersey','york','carolina','dakota','ohio','oklahoma','oregon','pennsylvania','rhode','carolina','dakota','tennessee','texas','utah' 'vermont','virginia' 'washington','virginia','wisconsin','wyoming','al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga', 'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy'])
    india=set(['india','indian','hindu','mumbai', 'delhi', 'bengalore', 'hyderabad', 'ahmedabad', 'chennai', 'kolkata', 'surat', 'pune', 'jaipur','Andhra','Pradesh','andhra','pradesh', 'amaravati', 'arunachal','pradesh', 'itanagar', 'assam', 'dispur', 'bihar', 'patna', 'chhattisgarh', 'raipur', 'goa', 'panaji', 'gujarat', 'gandhinagar', 'haryana', 'chandigarh', 'himachal','pradesh', 'shimla', 'jharkhand', 'ranch', 'karnataka', 'bengaluru', 'kerala', 'thiruvananthapuram', 'madhya', 'pradesh', 'bhopal', 'maharashtra', 'mumbai', 'manipur', 'imphal','meghalaya', 'shillong', 'mizoram', 'aizawl', 'nagaland', 'kohima', 'odisha', 'bhubaneswar', 'punjab', 'chandigarh', 'rajasthan', 'jaipur', 'sikkim', 'gangtok', 'tamil','nadu','chennai', 'telangana', 'hyderabad', 'tripura', 'agartala', 'uttar', 'pradesh', 'lucknow', 'uttarakhand', 'dehradun', 'gairsain', 'summer', 'bengal','kolkata'])
    if(len(set_in.intersection(us))>0):
        return "us"
    if(len(set_in.intersection(india))>0):
        return "ind"
    return "others"

In [133]:
a=set(['al'])
country(a)

'us'

In [134]:
data["country"]=data.cleaned_location.parallel_apply(country)

C:\Users\dadia\AppData\Local\Temp/ipykernel_12304/583345453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["country"]=data.cleaned_location.parallel_apply(country)


In [42]:
#data["country"].to_csv("C:/Users/Matt/xuexi/machine_learning/"+"location.csv")

In [135]:
data=data.dropna(subset=["country"])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10447 entries, 288 to 67905
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                10447 non-null  int64         
 1   user_name         10447 non-null  object        
 2   user_location     10447 non-null  object        
 3   user_description  10065 non-null  object        
 4   user_created      10447 non-null  object        
 5   user_followers    10447 non-null  int64         
 6   user_friends      10447 non-null  int64         
 7   user_favourites   10447 non-null  int64         
 8   user_verified     10447 non-null  bool          
 9   date              10447 non-null  object        
 10  text              10447 non-null  object        
 11  hashtags          7459 non-null   object        
 12  source            10445 non-null  object        
 13  retweets          10447 non-null  int64         
 14  favorites         10

# Graphs

In [136]:
ts=data.drop(labels=["date","text","user_location","user_followers","user_friends","user_verified","cleaned_text","user_name","user_description","user_created","user_favourites","hashtags","source","is_retweet","senti"],axis=1)
ts

id  retweets  favorites            new_date  \
288    1337899774788456448         0          1 2020-12-12 23:18:00   
274    1337948680263909380         1          2 2020-12-13 02:33:00   
264    1337974613280759808         0          0 2020-12-13 04:16:00   
259    1337979102544850946         0          6 2020-12-13 04:34:00   
248    1338017813374570496         0          0 2020-12-13 07:08:00   
...                    ...       ...        ...                 ...   
67914  1385233390841077760         0          0 2021-04-22 14:05:00   
67911  1385236168820854793         0          0 2021-04-22 14:16:00   
67907  1385236820854812676         0          0 2021-04-22 14:19:00   
67906  1385236855092822016         0          0 2021-04-22 14:19:00   
67905  1385236941180973057         0          0 2021-04-22 14:19:00   

                        cleaned_location country  
288               {gp, watford, partner}  others  
274            {malaysia, kuala, lumpur}  others  
264                      {mumbai, india}     ind  
259                         {delhi, new}     ind  
248                         {delhi, new}     ind  
...                                  ...     ...  
67914                          {germany}  others  
67911  {worldwide, canada, toronto, and}  others  
67907  {worldwide, canada, toronto, and}  others  
67906  {worldwide, canada, toronto, and}  others  
67905  {worldwide, canada, toronto, and}  others  

[10447 rows x 6 columns]

In [137]:
d=dict(ts['new_date'].value_counts())
a=list(d.keys())
b=list(d.values())
tmp=pd.DataFrame({"date":a,"num":b})
tmp=tmp.sort_values("date")

In [138]:
tmp

date  num
4884 2020-12-12 23:18:00    1
8072 2020-12-13 02:33:00    1
6432 2020-12-13 04:16:00    1
6433 2020-12-13 04:34:00    1
6434 2020-12-13 07:08:00    1
...                  ...  ...
596  2021-04-22 13:45:00    2
3408 2021-04-22 13:58:00    1
3409 2021-04-22 14:05:00    1
3410 2021-04-22 14:16:00    1
57   2021-04-22 14:19:00    3

[9534 rows x 2 columns]

In [139]:
new=pd.DataFrame(dict(tmp.set_index('date').resample('5D')["num"].sum()).items(),columns=['date','num'])
new

date   num
0  2020-12-12    62
1  2020-12-17    71
2  2020-12-22    49
3  2020-12-27    48
4  2021-01-01    88
5  2021-01-06    91
6  2021-01-11    81
7  2021-01-16   110
8  2021-01-21    70
9  2021-01-26    54
10 2021-01-31    30
11 2021-02-05   231
12 2021-02-10   314
13 2021-02-15   306
14 2021-02-20   278
15 2021-02-25   553
16 2021-03-02   447
17 2021-03-07   493
18 2021-03-12   715
19 2021-03-17   752
20 2021-03-22   609
21 2021-03-27   622
22 2021-04-01   520
23 2021-04-06  1036
24 2021-04-11  1345
25 2021-04-16   967
26 2021-04-21   505

In [140]:
new.to_csv(out_path+"intro_plot.csv",index=False)

In [141]:
x=np.array(new["date"])
y=np.array(new["num"])

plt.figure(figsize=(20, 5), dpi=100)
plt.plot(x,y)

plt.xlabel("Date")
plt.ylabel("Number of Tweets")
plt.title("Tweets per Day")

#ro_time=list(process_time("2020/8/11 00:00:00"),process_time())
plt.scatter(process_time("2020/8/9 00:00:00"),2078)
plt.annotate("Russian Vaccine Rollout",(process_time("2020/7/10 00:00:00"),2078),fontsize=9)

plt.scatter(process_time("2020/11/7 00:00:00"),3071)
plt.annotate("Pfizer annouces 90% effective rate",(process_time("2020/10/10 00:00:00"),3071),fontsize=9)

plt.scatter(process_time("2020/12/27 00:00:00"),13525)
plt.annotate("Pfizer rollout",(process_time("2020/12/10 00:00:00"),13525),fontsize=9)

plt.scatter(process_time("2021/1/11 00:00:00"),15049)
plt.annotate("Oxford/AstraZeneca rollout",(process_time("2020/12/10 00:00:00"),15049),fontsize=9)

plt.scatter(process_time("2021/1/26 00:00:00"),18117)
plt.annotate("Johnson & Johnson rollout",(process_time("2020/12/25 00:00:00"),18117),fontsize=9)

plt.scatter(process_time("2021/3/17 00:00:00"),10233)
plt.annotate("Biden Administration vaccine plans",(process_time("2021/2/15 00:00:00"),12000),fontsize=9)
plt.annotate("blood clots linked to Astazeneca",(process_time("2021/2/10 00:00:00"),11000),fontsize=9)

plt.scatter(process_time("2021/4/21 00:00:00"),4818)
plt.annotate("200 million vaccines administered in us",(process_time("2021/4/5 00:00:00"),4818),fontsize=9)

plt.scatter(process_time("2021/8/19 00:00:00"),6857)
plt.annotate("full approval for all age groups",(process_time("2021/8/1 00:00:00"),6857),fontsize=9)

plt.savefig(out_path+"Tweets per Day.png")